In [1]:
from datetime import datetime
import json
from pprint import pprint
from twitter_functions import * # these are the functions John wrote.
# from twitter_config import *

In [2]:
# Twitter credentials

auth =  { "consumer_key": "qVwvZDIaSoqCc1jAxIEtPB3Tq",
         "consumer_secret": "M29LjXVBcZtDwt1iB9UcT3CR7Q20I0HhpQlchTFLCCy2wv7c9D",
         "access_key": "419210344-7E1HtetDiPyAsm1T1HU5GPMIsSd0wRqpjGaX3Vsk",
         "access_secret": "MNlEbxiKhuDs6GqXu2u3s6iT876Q4B4bSegUpxMJ2kwAr"
       }

api = initialize_twitter(auth)

In [12]:
candidate_handles = ['ewarren','BernieSanders', 'Pete_Buttigieg', 'JoeBiden', 'AndrewYang', 'realDonaldTrump'] # my first group

for cand in candidate_handles:
    
    starting_user = [cand]
    print('-'*20)
    print(starting_user)
    
    ofile_name = (datetime.today().strftime("%Y%m%d") + "_" + 
                 starting_user[0] + "_" + # Just use the first starting_user for the name if there are multiple
                 "followers.txt")


    # We'll now go lookup the full information on your starting user(s). 
    starting_user_id = []

    # All records will be a dictionary with the twitter ID as the key and 
    # a UserRecord as the value. This is a named tuple I created. 
    all_records = lookup_users_from_handles(api, starting_user)

    # We need the IDs that we're getting followers from in a list. 
    for id in all_records : #access the keys, which are ids.
        starting_user_id.append(id)


    # How long is it going to take us to pull these followers?
    total_followers = 0
    for id, rec in all_records.items() :
        total_followers += rec.followers_count

    print("Ooh, {fol:,d} followers. A complete run with no limits run is ".format(fol=total_followers) + 
          "going to take {min:.2f} minutes ({hour:.2f} hours)".format(min=total_followers/5000,
                                                                      hour=total_followers/(60*5000)))
    
    # Now let's pull all the followers of our starting_user
    # the function I wrote allows you to cap the number of followers you pull
    # and uses the ID to generate the query.
    # 
    # Note that this pull is subject to rate limiting. You can make 15 calls per
    # 15 minute window and each can return 5000 users. 
    followers_of_starting = gather_followers(api,
                                             starting_user_id,
                                             follower_limit=None) # Modify this limit if you need to. 
                                                                  # Set it to "None" to get all   

    # followers_of_starting will be a dictionary with the key being the id(s) in starting_user_id
    # and the value is a list of all the followers.
    
    # And now we'll "hydrate" these user records.
    for start_id, list_of_followers in followers_of_starting.items() :

        # Using a set here instead of a list so that we pull each ID only once.
        ids_to_hydrate = {id for id in list_of_followers if id not in all_records}

        these_records = lookup_users_from_ids(api,ids=ids_to_hydrate)

        for id, rec in these_records.items() :
            all_records[id] = rec
            
            
            
    # Now let's write out all the records. I wrote some functions to help.
    with open(ofile_name,'w') as ofile :
        write_user_rec_headers(ofile)
        for id, rec in all_records.items() :
            write_user_rec(ofile, rec)


--------------------
['jordanmcnea']
Start lookup_users_from_ids on 1 handles.
20191220-155820: looking up user records for 1 handles.
20191220-155820:  users pulled:  1
total failures: 0
Ooh, 108 followers. A complete run with no limits run is going to take 0.02 minutes (0.00 hours)
Pulling followers for 419210344
Number pulled: 108
Start lookup_users_from_ids on 108 IDs.
20191220-155820: looking up user records for 100 IDs.
20191220-155821: looking up user records for 8 IDs.
20191220-155822:  users pulled:  108
total failures: 0
--------------------
['BrandonSnedigar']
Start lookup_users_from_ids on 1 handles.
20191220-155822: looking up user records for 1 handles.
20191220-155822:  users pulled:  1
total failures: 0
Ooh, 27 followers. A complete run with no limits run is going to take 0.01 minutes (0.00 hours)
Pulling followers for 1000037665708720133
Number pulled: 27
Start lookup_users_from_ids on 27 IDs.
20191220-155822: looking up user records for 27 IDs.
20191220-155823:  users